In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import h5py

In [2]:
matplotlib.rc('xtick', labelsize=28)
matplotlib.rc('ytick', labelsize=28)
matplotlib.rcParams['font.size']=28

In [3]:
data_path = "C:/Data/"

In [15]:
snap_path = data_path + '/run_CDM_L3N256_HY/snap_007.hdf5'
fof_path = data_path + '/run_CDM_L3N256_HY/fof_subhalo_tab_007.hdf5'
with h5py.File(fof_path, 'r') as s, h5py.File(snap_path, "r") as f:
    print(f['Header'].attrs['NumPart_Total'][4])
    print(f['Header'].attrs['NumPart_Total'][1])
    masses = f.get('PartType4/Masses')[()]

1290028
16777216


In [9]:
masses

array([1.86604292e-06, 1.12757478e-06, 1.64059200e-06, ...,
       1.77639831e-06, 1.67689051e-06, 1.36007911e-06])

In [4]:
def get_masses(sbh_pos, rbins, coord, mass_table):

    r = (coord[:,0] - sbh_pos[0])**2 + (coord[:,1] - sbh_pos[1])**2 + (coord[:,2] - sbh_pos[2])**2

    m_in = []
    for R in rbins:
        m_in.append(mass_table[r <= R].sum())
    return np.array(m_in)

In [ ]:
snap_path = data_path + '/run_CDM_L3N256_HY/snap_000.hdf5'
fof_path = data_path + '/run_CDM_L3N256_HY/fof_subhalo_tab_000.hdf5'

massive_partTypes = [0, 1, 4, 5]

max_no_subhalos = 5
with h5py.File(fof_path, 'r') as s, h5py.File(snap_path, "r") as f:
    masses = f.get('PartType0/Masses')[()]

In [10]:
masses

array([2.11230878e-06, 2.11230878e-06, 2.11230878e-06, ...,
       2.11230878e-06, 2.11230878e-06, 2.11230878e-06])

In [11]:
np.all(np.isclose(masses, masses[0]))

True